In [100]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys

In [8]:
feature_maps = (torch.randn(3, 256, 216, 184), torch.randn(3, 256, 108, 92), torch.randn(3, 256, 54, 46), torch.randn(3, 256, 27, 23))

In [6]:
rois = torch.Tensor([[  0.0000, 557.9707, 366.0170, 697.4088, 566.9425],
[  1.0000,   0.0000, 622.1016, 143.7075, 709.8333],
[  1.0000,  16.5338, 620.5057, 197.3975, 710.6006],
[  2.0000, 223.7691, 473.1749, 359.9460, 628.8488],
[  2.0000, 208.7294, 522.8946, 341.2042, 643.8867]])

In [11]:
rois_index = rois[:, 0].long()

In [13]:
rois_ = rois[:, 1:]

In [10]:
i = 0
curf = feature_maps[i]
scale_factor = 4 * (i + 1)

In [18]:
rois_rescale = torch.round(rois_ / scale_factor).long()

In [37]:
rois_index

tensor([0, 1, 1, 2, 2])

In [19]:
rois_rescale

tensor([[139,  92, 174, 142],
        [  0, 156,  36, 177],
        [  4, 155,  49, 178],
        [ 56, 118,  90, 157],
        [ 52, 131,  85, 161]])

In [80]:
oneD_coordinate = torch.arange(curf.size(2) * curf.size(3)).reshape(1, curf.size(2), curf.size(3))
y_coordinate = torch.floor(oneD_coordinate / curf.size(3))
x_coordinate = oneD_coordinate % curf.size(3)

In [81]:
oneD_coordinate

tensor([[[    0,     1,     2,  ...,   181,   182,   183],
         [  184,   185,   186,  ...,   365,   366,   367],
         [  368,   369,   370,  ...,   549,   550,   551],
         ...,
         [39192, 39193, 39194,  ..., 39373, 39374, 39375],
         [39376, 39377, 39378,  ..., 39557, 39558, 39559],
         [39560, 39561, 39562,  ..., 39741, 39742, 39743]]])

In [82]:
y_coordinate

tensor([[[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  2.,   2.,   2.,  ...,   2.,   2.,   2.],
         ...,
         [213., 213., 213.,  ..., 213., 213., 213.],
         [214., 214., 214.,  ..., 214., 214., 214.],
         [215., 215., 215.,  ..., 215., 215., 215.]]])

In [83]:
x_coordinate

tensor([[[  0,   1,   2,  ..., 181, 182, 183],
         [  0,   1,   2,  ..., 181, 182, 183],
         [  0,   1,   2,  ..., 181, 182, 183],
         ...,
         [  0,   1,   2,  ..., 181, 182, 183],
         [  0,   1,   2,  ..., 181, 182, 183],
         [  0,   1,   2,  ..., 181, 182, 183]]])

In [84]:
y_coordinate = y_coordinate.repeat(rois_rescale.size(0), 1, 1)
x_coordinate = x_coordinate.repeat(rois_rescale.size(0), 1, 1)

In [85]:
y_coordinate.size()

torch.Size([5, 216, 184])

In [86]:
one_mask = (x_coordinate >= rois_rescale[:, 0].reshape(-1, 1, 1)) & \
            (x_coordinate <= rois_rescale[:, 2].reshape(-1, 1, 1)) & \
            (y_coordinate >= rois_rescale[:, 1].reshape(-1, 1, 1)) & \
            (y_coordinate <= rois_rescale[:, 3].reshape(-1, 1, 1))

In [111]:
one_mask = one_mask.reshape(-1, 1, curf.size(2), curf.size(3))
one_mask.size()

torch.Size([5, 1, 216, 184])

In [123]:
feature_for_rois = curf[rois_index]
feature_for_rois.size()

torch.Size([5, 256, 216, 184])

In [105]:
curf.element_size() * curf.nelement()

122093568

In [108]:
# 过大，不能这么做，用其他办法，比如深度换广度
122093568 * 49 / 1073741824

5.57171630859375

In [128]:
one_mask[1][0][92]

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, 

In [125]:
masked = feature_for_rois.masked_fill_(~one_mask, 0)

In [129]:
masked[1][0][156]

tensor([-1.0190,  0.2505, -1.2973,  0.1266,  1.4590, -1.0186, -1.0845, -0.5185,
         0.2982, -0.9843, -0.0731,  0.5933, -1.0071,  0.2438,  1.2670, -0.1118,
         0.0731, -0.2255,  0.4128, -0.8701, -0.3399, -0.1652, -0.4243, -0.7005,
        -0.1433, -0.6158, -0.0508, -0.8184, -0.4700,  0.9004,  0.6842,  0.1634,
         0.5689, -2.2396, -0.7154,  1.4591,  0.5080,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.00

In [121]:
masked.size()

torch.Size([5, 256, 216, 184])

In [130]:
# 方案二：
# 把x的power的cumsum算出来之后再根据坐标相减
# e.g. 对x1 y1 x2 y2来说
# 他们的和等于的是
# cumsum[x2][y2] - cumsum[x1][y2] - cumsum[x2][y1] + cumsum[x1][y1]

In [145]:
p = nn.Parameter(torch.Tensor([3]))
minimumx = nn.Parameter(torch.Tensor([1e-6]), requires_grad=False)

In [146]:
xpower = torch.pow(torch.maximum(curf, minimumx), p)

In [150]:
xpower_cumsum = xpower.cumsum(2).cumsum(3)

In [156]:
x1 = rois_rescale[:, 0]
y1 = rois_rescale[:, 1]
x2 = rois_rescale[:, 2]
y2 = rois_rescale[:, 3]

In [157]:
x1.size()

torch.Size([5])

In [158]:
v1 = xpower_cumsum[rois_index, :, x1, y1]
v2 = xpower_cumsum[rois_index, :, x1, y2]
v3 = xpower_cumsum[rois_index, :, x2, y1]
v4 = xpower_cumsum[rois_index, :, x2, y2]

roi_power_sum = v4 - v3 - v2 + v1

In [159]:
roi_power_sum.size()

torch.Size([5, 256])

In [137]:
x_power_cumsum[0, 0, 139, 92]

tensor(-0.9566)

In [144]:
v1[0][0]

tensor(-0.9566)

In [160]:
x1

tensor([139,   0,   4,  56,  52])

In [161]:
y1

tensor([ 92, 156, 155, 118, 131])

In [177]:
w = x2 - x1
h = y2 - y1

In [211]:
winc = torch.ceil(w / 7).long()
hinc = torch.ceil(h / 7).long()

In [212]:
subrois_x1 = []
subrois_x2 = []
subrois_y1 = []
subrois_y2 = []
for i in range(7):
    cur_y1 = y1 + hinc * i
    if i < 6:
        cur_y2 = y1 + hinc * (i + 1) - 1
    else:
        cur_y2 = y2
    for j in range(7):
        cur_x1 = x1 + winc * j
        if j < 6:
            cur_x2 = x1 + winc * (j + 1) - 1
        else:
            cur_x2 = x2
            
        subrois_x1.append(cur_x1)
        subrois_y1.append(cur_y1)
        subrois_x2.append(cur_x2)
        subrois_y2.append(cur_y2)

In [213]:
sx1 = torch.cat(subrois_x1, dim=-1).reshape(7, 7, 5).permute(2, 1, 0)
sy1 = torch.cat(subrois_y1, dim=-1).reshape(7, 7, 5).permute(2, 1, 0)
sx2 = torch.cat(subrois_x2, dim=-1).reshape(7, 7, 5).permute(2, 1, 0)
sy2 = torch.cat(subrois_y2, dim=-1).reshape(7, 7, 5).permute(2, 1, 0)

In [214]:
rois_index_sub = rois_index.reshape(-1, 1, 1).repeat(1, 7, 7)

In [215]:
v1 = xpower_cumsum[rois_index_sub, :, sx1, sy1]

In [216]:
v1.size()

torch.Size([5, 7, 7, 256])